<a href="https://colab.research.google.com/github/Enrique-Albarran-IA/-horario-optimizado/blob/main/ch_Inferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
from dataclasses import dataclass
from typing import List, Set, Dict, Tuple
from copy import deepcopy
from collections import defaultdict

# ==================== Data Structures ====================
@dataclass
class CourseGroup:
    course_id: str
    group_id: str
    weekly_hours: int
    num_students: int

@dataclass
class Professor:
    id: str
    availability: Set[Tuple[str, int, int]]  # (day, start, end)
    courses: List[str]

@dataclass
class Classroom:
    id: str
    capacity: int

@dataclass
class Assignment:
    group: CourseGroup
    professor: Professor
    classroom: Classroom
    time_slots: List[Tuple[str, int, int]]

# ==================== Scheduler Class ====================
class Scheduler:
    def __init__(self,
                 courses: List[CourseGroup],
                 professors: List[Professor],
                 classrooms: List[Classroom],
                 time_slots: List[Tuple[str, int, int]]):
        self.courses = courses
        self.professors = professors
        self.classrooms = classrooms
        self.time_slots = time_slots
        self.HARD_PENALTY = 1000
        self.SOFT_PENALTY = 200

    # ==================== Core Algorithm ====================
    def generate_initial_solution(self) -> List[Assignment]:
        solution = []
        for group in self.courses:
            # Professor selection with validation
            eligible_profs = [p for p in self.professors
                             if group.course_id in p.courses]
            if not eligible_profs:
                raise ValueError(f"No professors available for {group.course_id}")
            prof = random.choice(eligible_profs)

            # Classroom selection with validation
            eligible_rooms = [r for r in self.classrooms
                             if r.capacity >= group.num_students]
            if not eligible_rooms:
                raise ValueError(f"No classroom for {group.num_students} students")
            room = random.choice(eligible_rooms)

            # Time slot selection with fallback
            valid_slots = [slot for slot in self.time_slots
                          if slot in prof.availability]
            if len(valid_slots) < group.weekly_hours:
                # Try other professors first
                remaining_profs = [p for p in eligible_profs if p != prof]
                if remaining_profs:
                    return self.generate_initial_solution()  # Retry recursively
                else:  # Fallback to random slots
                    valid_slots += random.sample(self.time_slots,
                                               group.weekly_hours - len(valid_slots))

            slots = random.sample(valid_slots, group.weekly_hours)
            solution.append(Assignment(
                group=deepcopy(group),
                professor=deepcopy(prof),
                classroom=deepcopy(room),
                time_slots=slots
            ))
        return solution

    def calculate_penalty(self, solution: List[Assignment]) -> int:
        penalty = 0
        classroom_usage = defaultdict(set)  # {(day, hour): {classroom_ids}}
        professor_usage = defaultdict(set)  # {(day, hour): {professor_ids}}

        for assignment in solution:
            # Hard constraint: classroom capacity
            if assignment.classroom.capacity < assignment.group.num_students:
                penalty += self.HARD_PENALTY

            # Track hours for each time slot
            for slot in assignment.time_slots:
                day, start, end = slot
                # Check each hour in the time slot
                for hour in range(start, end):
                    time_key = (day, hour)

                    # Classroom conflict
                    if assignment.classroom.id in classroom_usage[time_key]:
                        penalty += self.HARD_PENALTY
                    classroom_usage[time_key].add(assignment.classroom.id)

                    # Professor conflict
                    if assignment.professor.id in professor_usage[time_key]:
                        penalty += self.HARD_PENALTY
                    professor_usage[time_key].add(assignment.professor.id)

                    # Professor availability
                    if not any(slot_day == day and slot_start <= hour < slot_end
                              for (slot_day, slot_start, slot_end) in assignment.professor.availability):
                        penalty += self.HARD_PENALTY

            # Soft constraint: consecutive hours preference
            day_hours = defaultdict(list)
            for slot in assignment.time_slots:
                day, start, _ = slot
                day_hours[day].append(start)
            for day, hours in day_hours.items():
                sorted_hours = sorted(hours)
                # Penalize gaps between classes
                for i in range(1, len(sorted_hours)):
                    if sorted_hours[i] - sorted_hours[i-1] > 1:
                        penalty += self.SOFT_PENALTY

            # Soft constraint: multiple days
            if len({slot[0] for slot in assignment.time_slots}) > 1:
                penalty += self.SOFT_PENALTY * 2

        return penalty

    def mutate(self, solution: List[Assignment]) -> List[Assignment]:
        new_solution = deepcopy(solution)

        # Target assignments with highest individual penalty
        penalties = [self.calculate_penalty([a]) for a in new_solution]
        idx = random.choices(
            range(len(new_solution)),
            weights=penalties,
            k=1
        )[0]
        target = new_solution[idx]

        mutation_type = random.random()
        if mutation_type < 0.4:  # Change time slots
            valid_slots = [slot for slot in self.time_slots
                          if slot in target.professor.availability]
            if valid_slots:
                new_slots = random.sample(valid_slots, target.group.weekly_hours)
                target.time_slots = new_slots
        elif mutation_type < 0.7:  # Change professor
            eligible = [p for p in self.professors
                       if target.group.course_id in p.courses]
            if len(eligible) > 1:
                target.professor = deepcopy(random.choice(
                    [p for p in eligible if p.id != target.professor.id]
                ))
        else:  # Change classroom
            eligible = [r for r in self.classrooms
                       if r.capacity >= target.group.num_students]
            if len(eligible) > 1:
                target.classroom = deepcopy(random.choice(
                    [r for r in eligible if r.id != target.classroom.id]
                ))

        return new_solution

    def simulated_annealing(self,
                            initial_solution: List[Assignment],
                            iterations: int = 2000,
                            verbose: bool = True) -> Tuple[List[Assignment], int]:
        current = deepcopy(initial_solution)
        current_penalty = self.calculate_penalty(current)
        best = deepcopy(current)
        best_penalty = current_penalty

        initial_temp = 1.0
        temp = initial_temp
        cooling_rate = 0.95

        for i in range(iterations):
            neighbor = self.mutate(current)
            neighbor_penalty = self.calculate_penalty(neighbor)

            # Acceptance probability
            if neighbor_penalty < current_penalty or \
               random.random() < math.exp((current_penalty - neighbor_penalty) / temp):
                current = neighbor
                current_penalty = neighbor_penalty

                if neighbor_penalty < best_penalty:
                    best = deepcopy(neighbor)
                    best_penalty = neighbor_penalty

            # Dynamic cooling
            temp = initial_temp * math.exp(-cooling_rate * (i / iterations))

            if verbose and i % 250 == 0:
                print(f"Iteration {i}: Temp {temp:.2f} | Best Penalty {best_penalty}")

        return best, best_penalty

    # ==================== Utility Methods ====================
    def print_schedule(self, solution: List[Assignment]):
        for assignment in solution:
            print(f"\nCourse: {assignment.group.course_id} ({assignment.group.group_id})")
            print(f"Professor: {assignment.professor.id}")
            print(f"Classroom: {assignment.classroom.id} (Cap: {assignment.classroom.capacity})")
            print(f"Students: {assignment.group.num_students}")
            print("Schedule:")
            for day, start, end in sorted(assignment.time_slots):
                print(f"  {day} {start:02d}:00-{end:02d}:00")

# ==================== Sample Data & Execution ====================
if __name__ == "__main__":
    # Sample data setup
    time_slots = [
        ('Mon', 9, 10), ('Mon', 10, 11), ('Mon', 11, 12), ('Mon', 12, 13),
        ('Mon', 13, 14), ('Mon', 14, 15), ('Mon', 15, 16), ('Mon', 16, 17),
        ('Tue', 9, 10), ('Tue', 10, 11), ('Tue', 11, 12), ('Tue', 12, 13),
        ('Tue', 13, 14), ('Tue', 14, 15), ('Tue', 15, 16), ('Tue', 16, 17),
        ('Wed', 9, 10), ('Wed', 10, 11), ('Wed', 11, 12), ('Wed', 12, 13),
        ('Wed', 13, 14), ('Wed', 14, 15), ('Wed', 15, 16), ('Wed', 16, 17),
        ('Thu', 9, 10), ('Thu', 10, 11), ('Thu', 11, 12), ('Thu', 12, 13),
        ('Thu', 13, 14), ('Thu', 14, 15), ('Thu', 15, 16), ('Thu', 16, 17),
        ('Fri', 9, 10), ('Fri', 10, 11), ('Fri', 11, 12), ('Fri', 12, 13),
        ('Fri', 13, 14), ('Fri', 14, 15), ('Fri', 15, 16), ('Fri', 16, 17)
    ]

    courses = [
        CourseGroup("Redes, Arboles y Grafos", "s1", 4, 40),
        CourseGroup("Manejo de datos Masivos", "s3", 5, 35),
        CourseGroup("Aprendizaje Automatico", "s2", 5, 35),
        CourseGroup("Sistemas Paralelos", "s4", 6, 35),
        CourseGroup("Busqueda de soluciones e inferencia Bayesiana", "s5", 5, 35),
        CourseGroup("Seminario", "s6", 5, 35),
        CourseGroup("Procesamiento Digital de Imagenes", "s7", 5, 35)
    ]

    professors = [
        Professor("Arzate", {('Mon', 9, 10), ('Mon', 10, 11), ('Tue', 9, 10),
                            ('Tue', 10, 11), ('Wed', 9, 10), ('Thu', 9, 10)},
                ["Aprendizaje Automatico"]),
        Professor("Hermosillo", {('Mon', 14, 15), ('Wed', 10, 11),
                                ('Wed', 11, 12), ('Thu', 14, 15),
                                ('Fri', 14, 15), ('Fri', 15, 16)},
                ["Busqueda de soluciones e inferencia Bayesiana"]),
        Professor("Rendon", {('Tue', 11, 12), ('Tue', 12, 13),
                            ('Wed', 13, 14), ('Thu', 11, 12),
                            ('Fri', 11, 12), ('Fri', 12, 13)},
                ["Procesamiento Digital de Imagenes"]),
        Professor("Kevin", {('Mon', 15, 16), ('Mon', 16, 17), ('Mon', 17, 18),
                           ('Tue', 15, 16), ('Tue', 16, 17), ('Tue', 17, 18),
                           ('Wed', 15, 16), ('Wed', 16, 17),
                           ('Thu', 15, 16), ('Thu', 16, 17),
                           ('Fri', 15, 16), ('Fri', 16, 17)},
                ["Manejo de datos Masivos", "Sistemas Paralelos"]),
        Professor("Omar", {('Mon', 12, 13), ('Mon', 13, 14),
                          ('Tue', 12, 13), ('Wed', 12, 13),
                          ('Thu', 12, 13), ('Fri', 12, 13)},
                ["Seminario"]),
        Professor("Dan", {('Mon', 9, 10), ('Mon', 10, 11),
                         ('Tue', 9, 10), ('Tue', 10, 11),
                         ('Wed', 9, 10), ('Thu', 9, 10)},
                ["Redes, Arboles y Grafos"]),
    ]

    classrooms = [
        Classroom("R1", 60),
        Classroom("R2", 45)
    ]

    # Initialize and run scheduler
    scheduler = Scheduler(courses, professors, classrooms, time_slots)
    initial = scheduler.generate_initial_solution()
    print(f"Initial penalty: {scheduler.calculate_penalty(initial)}")

    best_solution, best_penalty = scheduler.simulated_annealing(initial)
    print(f"\nFinal penalty: {best_penalty}")
    scheduler.print_schedule(best_solution)

Initial penalty: 6800
Iteration 0: Temp 1.00 | Best Penalty 6800
Iteration 250: Temp 0.89 | Best Penalty 3800
Iteration 500: Temp 0.79 | Best Penalty 3800
Iteration 750: Temp 0.70 | Best Penalty 3800
Iteration 1000: Temp 0.62 | Best Penalty 3800
Iteration 1250: Temp 0.55 | Best Penalty 3800
Iteration 1500: Temp 0.49 | Best Penalty 3800
Iteration 1750: Temp 0.44 | Best Penalty 3800

Final penalty: 3800

Course: Redes, Arboles y Grafos (s1)
Professor: Dan
Classroom: R2 (Cap: 45)
Students: 40
Schedule:
  Mon 09:00-10:00
  Mon 10:00-11:00
  Tue 09:00-10:00
  Tue 10:00-11:00

Course: Manejo de datos Masivos (s3)
Professor: Kevin
Classroom: R1 (Cap: 60)
Students: 35
Schedule:
  Fri 15:00-16:00
  Mon 15:00-16:00
  Thu 16:00-17:00
  Tue 15:00-16:00
  Tue 16:00-17:00

Course: Aprendizaje Automatico (s2)
Professor: Arzate
Classroom: R1 (Cap: 60)
Students: 35
Schedule:
  Mon 10:00-11:00
  Thu 09:00-10:00
  Tue 09:00-10:00
  Tue 10:00-11:00
  Wed 09:00-10:00

Course: Sistemas Paralelos (s4)
Profes